
# Getting Started with AMC Insights
A starting point for new users to the service. Contains a walkthrough of onboarding your first client as well as creating and invoking AMC workflows.

This guide (as well as the other notebooks in the repository) are set to **read-only** by default. It is recommended to copy the file and work from the copy if you would like to save your work.

#### Import Libraries

In [ ]:
import json
import os

from dotenv import load_dotenv

from library.workflow_interface import WorkflowInterface
from library.customer_interface import CustomerInterface

#### Define Global Configurations
Run the cell below to initialize the interface modules in order to interact with the correct architecture resources.

In [ ]:
# Defined in the ddk.json file and set in the instance lifecycle configuration
load_dotenv()
TEAM_NAME = os.environ['TEAM_NAME']
ENV = os.environ['ENV']

# Setting these interface classes allow calling the various WFM and TPS methods
workflow_interface = WorkflowInterface(TEAM_NAME=TEAM_NAME, ENV=ENV)
customer_interface = CustomerInterface(TEAM_NAME=TEAM_NAME, ENV=ENV)

#### Step 1: Onboard A New Customer

The Tenant Provisioning Service (TPS) is used for onboarding new customers to each team space. Each customer is tied 1 to 1 with an AMC instance.

The following notebook cells will define your customer AMC instance details and onboard your customer.

##### Define Customer Details
Edit and run the cell below to define your customer details.

In [ ]:
# This is the same value passed in the data_pipeline_parameters 'dataset' of the ddk.json file
dataset_name = "<INSERT DATASET NAME>" 

# The AMC Instance information can be found on your AMC UI page
amc_api_endpoint = "<ENTER AMC API ENDPOINT URL>"
amc_s3_bucket_name = "<ENTER AMC S3 BUCKET NAME>"
amc_data_upload_acct = "<ENTER AMC DATA UPLOAD ACCOUNT ID>"

## The AMC Instance AWS region (found in the API Endpoint URL) e.g. 'us-east-1'
# This should be the same as your deployment region
region = "<ENTER AMC INSTANCE REGION>"

## OPTIONAL: Change to your desired Customer ID (keep less than 25 characters) 
# This value must be unique for each customer
customerId = "democustomer"

# OPTIONAL: Change to your desired Customer Name
customerName = "Customer"

### Customer Prefix and Endemic Type are used to group together similar customers with overlapping workflows
## These are used in the Workflow Library to facilitate management across multiple customers at once
# OPTIONAL: Change to your desired Customer Prefix
customerPrefix = "demo"
# OPTIONAL: Can Be <ENDEMIC or NON-ENDEMIC>
endemicType = "ENDEMIC"


##### Confirm Customer Details

Run the cell below to format the defined customer settings.

In [ ]:
customerDetails = {
    "customerPrefix": customerPrefix,
    "customerName": customerName, 
    "endemicType": endemicType, 
    "region": region,
    "amc":{
        "amcDatasetName": dataset_name,
        "amcApiEndpoint": amc_api_endpoint,
        "amcOrangeAwsAccount": amc_data_upload_acct,
        "amcS3BucketName": amc_s3_bucket_name
    }
}

print(f'customerId = {customerId}')
print(f'customerDetails = {json.dumps(customerDetails, indent=4)}')

##### Submit Customer Onboarding

Run the cell below to start the process of onboarding your customer and creating the AMC S3 Bucket if it does not already exist.

In [ ]:
customer_interface.onboard_customer(
    customerId=customerId, 
    customerDetails=customerDetails
    )

_Wait a few minutes for the cloudformation stack to be deployed BEFORE moving to Step #2._

_You Can Verify the Status by going to AWS Step Functions and waiting until the state machine named tps-&lt;TeamName&gt;-initialize-amc has 1 Succeeded Execution Status._  
  
**The remaining steps require data to be present in your AMC Instance. Ensure that you have data in your account before continuing.**

#### Step 2: Create a Workflow Record 

The Workflow Manager Service (WFM) is used to manage and schedule AMC workflows. The following steps  will walk through the process of creating a new workflow record.

The query used here is the Time To Conversion Query from the Interactive Query Library (IQL) in the AMC UI.

The query finds out how long it takes for your customers to convert after last seeing your ad. You can use this information to adjust the duration of campaign and promotion to maximize sales. In our amazon_attributed_events_by_conversion_time and amazon_attributed_events_by_traffic_time tables, we report up to 14 days after the customers’ last exposure to your ad.

##### Define Workflow Query

Run the cell below to define the query that will be used in the workflow.


In [ ]:
amc_query = """
SELECT
      advertiser,
      campaign,
        ( 
            CASE WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 60 THEN
                '1 | < 1 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 600 THEN
                '2 | 1 - 10 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 1800 THEN
                '3 | 10 - 30 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 3600 THEN
                '4 | 30 - 60 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 7200 THEN
                '5 | 1 - 2 HRS'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 43200 THEN
                '6 | 2 - 12 HRS'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 86400 THEN
                '7 | 12 - 24 HRS'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 604800 THEN
                '8 | 1 - 7 DAYS'
            ELSE
                '9 | 7+ DAYS'
END
) AS time_to_conversion,
        SUM(purchases) AS purchases,
        SUM(total_purchases) AS total_brand_purchases
FROM
    amazon_attributed_events_by_conversion_time
    
GROUP BY 1,2,3
"""

print (amc_query)

__NOTE__: This is just one example of a workflow query you can run on your AMC Instance. Refer to the Interactive Query Library (IQL) in the AMC UI for a list of other queries for different use cases. Queries can also be customized for you unique use case as well.

##### Define Workflow Details

Run the cell below to define the workflow settings.

In [ ]:
## OPTIONAL: Change to your desired Workflow ID
# This value must be unique for each workflow per customer (workflowId + customerId = composite key)
workflowId = "time_to_conversion_v1"

# OPTIONAL: Change to your desired Workflow Name
workflowName = "time_to_conversion"

# OPTIONAL: Change to your desired Workflow Description
workflowDescription = "Runs the time_to_conversion workflow."

### Default Payload
## The following parameters are saved to the Workflow and will be used in the absense of invocation overrides
# OPTIONAL: Change to your desired Time Window End
timeWindowEnd = "today(-1)"

# OPTIONAL: Change to your desired Time Window Start 
timeWindowStart = "today(-91)"

# OPTIONAL: Change to your desired Time Window Type
timeWindowType = "EXPLICIT"

# OPTIONAL: Change to your desired Workflow Executed Date
workflowExecutedDate = "now()"


##### Confirm Workflow Details

Run the cell below to format the defined workflow settings.

In [ ]:
workflowId = workflowId
workflowDetails = {
    "metadata": {
        "workflowName": workflowName,
        "description": workflowDescription,
    },
    "sqlQuery": amc_query,
    "defaultPayload": {
            "timeWindowEnd": timeWindowEnd,
            "timeWindowStart": timeWindowStart,
            "timeWindowType": timeWindowType,
            "workflowExecutedDate": workflowExecutedDate
    }
}

print(f'workflowId = {workflowId}')
print(f'workflowDetails = {json.dumps(workflowDetails, indent=4)}')

##### Submit Workflow Create

Run the cell below to create and save the workflow details.

In [ ]:
workflow_interface.set_workflow_record(
    workflowId=workflowId,
    customerId=customerId,
    workflowDetails=workflowDetails
    )

#### Step 3: Invoke the Workflow to be Executed Ad Hoc

Saved workflows can be executed adhoc or on a schedule. In the following step we will adhoc invoke the workflow we just created.

##### Submit Workflow Invoke Request

Run the cell below to submit the workflow to be processed once.

In [ ]:
workflow_interface.invoke_workflow(
    workflowId=workflowId, 
    customerId=customerId
    )

#### Your workflow is now being processed and executed
Once the workflow has run, data will be uploaded to your AMC S3 Bucket with the results of the workflow query and then processed through the data lake.

The execution status of the workflow can be viewed in the AMCExecutionStatus table.

##### View Execution Status

Run the cell below to view this table.

In [ ]:
workflow_interface.get_execution_status()

#### Step 4: Schedule the Workflow

Workflows can also be configured to automatically run on a set schedule. Optionally passing in 'payload' values allows you to override the saved defaultParameters on the workflow.

The steps below will walk through deploying a schedule without payload overrides on the workflow created earlier.

##### Define Schedule Details

Run the cell below to define the workflow schedule.

In [ ]:
## OPTIONAL: Change to your desired Schedule Name
# This value must be unique for each schedule per customer (scheduleName + customerId = composite key)
scheduleName = "time_to_conversion_daily"

# OPTIONAL: Can be set to 'ENABLED' or 'DISABLED'
state = "ENABLED"

#### OPTIONAL: Change to your desired schedule frequency
### Format is 'custom({H/D/W/M} {Day of the week/month} {Hour of the day})'
## E.g. 
#   custom(H * *) will be run every hour
#   custom(W 2 8) will be run weekly, on the 2nd day of the week, between 8:00 and 8:59 UTC
#   custom(D * 14) will be run daily, between 14:00 and 14:59 UTC
#   custom(M 15 2) will be run monthly, on the 15th day of the month, between 2:00 and 2:59 UTC
scheduleExpression = "custom(D * *)"

##### Confirm Workflow Schedule

Run the cell below to format the defined schedule settings.

In [ ]:
customerId = customerId
workflowId = workflowId
scheduleDetails = {
    "scheduleName": scheduleName,
    "state": state,
    "scheduleExpression": scheduleExpression
}

print(f'customerId = {customerId}')
print(f'workflowId = {workflowId}')
print(f'scheduleDetails = {scheduleDetails}')

##### Submit Workflow Schedule

Run the cell below to deploy the workflow schedule.

In [ ]:
workflow_interface.set_workflow_schedule(
    workflowId=workflowId,
    customerId=customerId,
    scheduleDetails=scheduleDetails
)

#### Your workflow is now scheduled

Workflows can also be scheduled using cron() statements instead of the custom() method used in the cells above. E.g. scheduleExpression = 'cron(15 10 * * ? *)'

It is recommended, however, to use custom() as it utilizes less resources.

#### Step 5: Create a Workflow Library Record

The Workflow Library provides a quick and easy way to manage and deploy workflows/schedules to multiple customers at once. The following steps walk through this proccess on a new workflow.

##### Define Worklow Details

Run the cell below to define the workflow details.

In [ ]:
sqlQuery = """
    WITH IMP_CLICKSSEG AS (   
            SELECT  A.ADVERTISER, 
                    A.behavior_segment_name AS SEGMENT, 
                    SUM(A.total_cost/100000) AS impression_cost,
                    behavior_segment_matched AS MATCHED_SEGMENT, 
                    SUM(A.IMPRESSIONS) AS IMPRESSIONS, 
                    SUM(B.clicks) AS CLICKS, 
                    COUNT(DISTINCT A.USER_ID) AS REACH 
            FROM    display_impressions_by_user_segments A 
                LEFT JOIN   DISPLAY_CLICKS B 
                    ON  A.request_tag = B.request_tag 
            GROUP BY    A.ADVERTISER, 
                        A.behavior_segment_name, 
                        A.behavior_segment_matched
            ), 
    USR_SEG_IMP AS (   
            SELECT  user_id, 
                    advertiser_id, 
                    behavior_segment_name, 
                    behavior_segment_matched, 
                    SUM(total_cost)/100000 AS impression_cost 
            FROM    display_impressions_by_user_segments
            GROUP BY    user_id, 
                        advertiser_id, 
                        behavior_segment_name, 
                        behavior_segment_matched 
            ), 
    CONV_SEG AS ( 
            SELECT  A.ADVERTISER, 
                    A.tracked_asin, 
                    B.behavior_segment_name AS SEGMENT, 
                    B.behavior_segment_matched AS MATCHED_SEGMENT, 
                    SUM(A.CONVERSIONS) AS PURCHASES, 
                    SUM(A.product_sales) AS sales_tracked, 
                    SUM(A.total_product_sales) AS sales_tracked_brand, 
                    ROUND(SUM(A.total_product_sales)/SUM(B.impression_cost),2) AS roas_brand, 
                    ROUND(SUM(A.product_sales)/SUM(B.impression_cost),2) AS roas_salestracked, 
                    SUM(B.impression_cost) AS total_cost_fromconvtable 
            FROM    amazon_attributed_events_by_conversion_time A 
                INNER JOIN USR_SEG_IMP B 
                    ON  A.USER_ID = B.USER_ID 
                        AND A.ADVERTISER_ID = B.ADVERTISER_ID 
            WHERE conversion_event_subtype = 'order' 
            GROUP BY    A.ADVERTISER, 
                        A.tracked_asin, 
                        B.behavior_segment_name, 
                        B.behavior_segment_matched 
            ) 
    SELECT  A.ADVERTISER, 
            BUILT_IN_PARAMETER('TIME_WINDOW_START') AS time_window_start, 
            BUILT_IN_PARAMETER('TIME_WINDOW_END') AS time_window_end, 
            A.SEGMENT, A.MATCHED_SEGMENT, 
            A.IMPRESSIONS, 
            A.REACH, 
            B.tracked_asin, 
            B.PURCHASES, 
            B.SALES_TRACKED, 
            B.sales_tracked_brand, 
            A.CLICKS, 
            roas_brand, 
            roas_salestracked, 
            A.impression_cost, 
            (B.PURCHASES/A.REACH) AS conversion_rate_perc, 
            B.total_cost_fromconvtable 
    FROM IMP_CLICKSSEG A 
    LEFT JOIN CONV_SEG B 
        ON A.ADVERTISER = B.ADVERTISER 
            AND A.SEGMENT = B.SEGMENT 
            AND A.MATCHED_SEGMENT = B.MATCHED_SEGMENT"
"""

# OPTIONAL: Change to your desired Workflow ID
workflowId = "exposure_group_analysis_v1"

# OPTIONAL: Change to your desired Workflow Name
workflowName = "exposure_group_analysis"

# OPTIONAL: Change to your desired Workflow Description
workflowDescription = "runs the exposure_group_analysis workflow"

### Default Payload
## The following parameters are saved to the Workflow and will be used in the absense of invocation overrides
# OPTIONAL: Change to your desired Time Window Start
timeWindowEnd = "today(-1)"

# OPTIONAL: Change to your desired Time Window Start
timeWindowStart = "today(-8)"

# OPTIONAL: Change to your desired Time Window Type
timeWindowType = "EXPLICIT"

# OPTIONAL: Change to your desired Workflow Executed Date
workflowExecutedDate = "now()"

##### Define Schedule Details

A workflow schedule can optionally be passed in as well in order to deploy a schedule along with the workflow. If 'schedule' is not passed in as a parameter then the worklow will be deployed without a schedule.

Run the cell below to define the workflow schedule.

In [ ]:
# OPTIONAL: Change to your desired Schedule Name
scheduleName = "exposure_group_analysis_weekly"

# OPTIONAL: Can be set to 'ENABLED' or 'DISABLED'
state = "ENABLED"

#### OPTIONAL: Change to your desired schedule frequency
scheduleExpression = "custom(W 1 11)"

##### Define Customer Details

When defining a workflow library record, you can optionally pass in parameters for 'endemicType' and 'customerPrefix'. The workflow and schedule will only be deployed for customers that share the same endemicType and/or customerPrefix to the workflow library record.

If no values are passed in, the workflow and schedule will be deployed for all customers.

In this demo we will deploy the workflow and schedule to all customers who are set to 'ENDEMIC'.

In [ ]:
endemicType = "ENDEMIC"

##### Confirm Workflow Library Record

Run the cell below to format the defined workflow library record.

In [ ]:
workflowId = workflowId
endemicType = endemicType
workflowDetails = {
    "metadata": {
        "workflowName": workflowName,
        "description": workflowDescription
    },
    "sqlQuery": sqlQuery,
    "defaultPayload": {
        "timeWindowEnd": timeWindowEnd,
        "timeWindowStart": timeWindowStart,
        "timeWindowType": timeWindowType,
        "workflowExecutedDate": workflowExecutedDate
    }
}
schedule = {
    "scheduleName": scheduleName,
    "scheduleDetails": {
        "state": state,
        "scheduleExpression": scheduleExpression
    }
}

print(f'workflowId = {workflowId}')
print(f'endemicType = {endemicType}')
print(f'workflowDetails = {workflowDetails}')

##### Submit Workflow Library Record

Run the cell below to submit and deploy the defined workflow and schedule.

In [ ]:
workflow_interface.set_workflow_library_record(
    workflowId=workflowId,
    endemicType=endemicType,
    workflowDetails=workflowDetails,
    schedule=schedule
)

#### Your workflow and schedule are now deployed

Re-submitting workflows from the library table will allow you to deploy changes to all customers without needing to individually update each record.

#### Step 6: Delete Workflow Schedules and Records

Run the cells below to remove all of the workflow records created during this walkthrough.

In [ ]:
workflow_interface.delete_workflow_schedule(
    customerId=customerId,
    scheduleName="time_to_conversion_daily"
)

workflow_interface.delete_workflow_record(
    customerId=customerId,
    workflowId="time_to_conversion_v1"
)

workflow_interface.delete_workflow_library_record(
    workflowId='exposure_group_analysis_v1'
)

#### **Next Steps**
After going through the guide above, the following notebooks provide templates for interacting with the microservice solution.

**Customers**: managing and onboarding new customers to the service.  
**Workflows**: managing and invoking workflows for single customers.  
**Workflow_Schedules**: managing workflow schedules for single customers.  
**Workflow_Library**: managing workflow and schedules for multiple customers.  

The reference folder located in this deployed instance contains additional useful documentation.